# Introduction to the SecML library

In this tutorial, you will learn the basic components of the [SecML library](https://github.com/pralab/secml).
At the end of the exercise, you will be familiar with the core data structure of the library, the [CArray](https://secml.readthedocs.io/en/v0.15/secml.array.html), and how to import pre-trained machine learning models from [scikit-learn](https://scikit-learn.org/stable/index.html) and [PyTorch](https://pytorch.org) (with a brief tutorial on it).



## Installation

Before installing SecML, we strongly suggest to create an environment where to download all the dependancies of the library through [Anaconda Python](https://docs.conda.io/en/latest/miniconda.html). Follow the linked page to install Miniconda (a minimal version of Conda).
After the installation, you can create a *conda environment* from your command line:

```bash
conda create -n secml python=3.8
```

Once the environment has been installed, you can activate it and install SecML:

```bash
conda activate secml
python -m pip install "secml[pytorch,foolbox]"
python -m install notebook
```

Once the procedure is complete, you can verify that SecML is correctly installed inside your environment.
Open a Python interpreter and type:
```python
import secml
print(secml.__version__)
```

Restart the notebook inside the conda environment to continue the exercise. Execute the following code to ensure you can access SecML inside the notebook.

In [ ]:
import secml
import foolbox
import sklearn.datasets
import torch
print('SecML:', secml.__version__)
print('Foolbox:', foolbox.__version__)
print('PyTorch:', torch.__version__)

# Part 1 - CArray: the basic data structure

The CArray is the base class that is used inside SecML to create vectors and matrices.
If you are already familiar with NumPy, you will recognize many functions and helpers along the tutorial.

In [ ]:
from secml.array import CArray

x = CArray([0,1,2,3])
print(x)
print('Shape of single-row vector: ', x.shape)
x = CArray([[0,1,2,3], [4,5,6,7]])
print(x)
print('Shape of 2D vector:', x.shape)

You can perform basic mathematical operations between CArrays:

In [ ]:
x1 = CArray([1,2,3,4])
x2 = CArray([5,6,7,8])

print(x1 + x2) # Element-wise sum
print(x1 - x2) # Element-wise subtraction
print(x1 * x2) # Element-wise multiplication
print(x1 / x2) # Element-wise division
print(x1.dot(x2)) # Dot product
print(x1.T) # Transpose
print(x1.norm(order=2)) # Compute norm

You can perform operations between 2D vectors:

In [ ]:
x1 = CArray([[1,2,3,4],[1,2,3,4]])
x2 = CArray([[5,6,7,8], [5,6,7,8]])

print(x1 + x2) # Element-wise sum
print(x1 - x2) # Element-wise subtraction
print(x1 * x2) # Element-wise multiplication
print(x1 / x2) # Element-wise division
print(x1.T.dot(x2)) # Dot product between (4,2) and (2,4) matrices
print(x1.norm_2d(order=2, axis=0)) # Norm of each column
print(x1.norm_2d(order=2, axis=1)) # Norm of each row
print(x1.flatten(), x1.flatten().shape) # Flatten the matrix to one single row

You can import data from numpy, by passing a numpy array to the CArray constructor:

In [ ]:
import numpy as np

x = np.array([0,1,2,3])
print('Numpy array:', x, 'with type', type(x))
x = CArray(x)
print('CArray of numpy array:', x, 'with type', type(x))
x = x.tondarray()
print('Back to ', type(x))

The CArray class offers helper functions to create data from known distributions, like the [Normal Distribution](https://en.wikipedia.org/wiki/Normal_distribution):

In [ ]:
x = CArray.randn((3,3)) # normal distribution
print(x)
x = CArray.zeros((2,5)) # 2D vector with only zeros
print(x)
x = CArray.ones((3,3)) # 2D vector with only ones
print(x)
x = CArray.eye(4,4)
print(x)

**PLEASE REMARK** that the CArray class only supports **2D** data. Passing a high-dimensional data shape will result in a flatten:

In [ ]:
x = np.random.rand(10,10,10)
xc = CArray(x)
print('NumPy shape:', x.shape)
print('CArray shape:', xc.shape)

# Exercise 1
Use the code above to complete the assignment.
* Create two CArray from the normal distribution with shape (5,6)
* Compute the dot product of the two newly-created CArray
* Flatten the result and compute the euclidean norm (which order?)
* Create an identity of shape (5,5) and a 2D vectors of zeros with shape (5,5)
* Sum and multiply the two newly-created CArray

# Part 2 - Import classifiers inside SecML

The SecML library offers wrappers for PyTorch and scikit-learn models.
More details on the creation and training of models inside SecML can be found on the [GitHub repository](https://github.com/pralab/secml/tree/master/tutorials). Wrapping a model is easy: the library offers classes that accepts models from the desired framework.

In [ ]:
# Wrapping a scikit-learn classifier
from sklearn.svm import SVC
from secml.ml.classifiers import CClassifierSkLearn
model = SVC()
secml_model = CClassifierSkLearn(model)

Models can be pre-trained (as the one in PyTorch), and they can also be trained *inside* SecML.

In [ ]:
import sklearn

X, y = sklearn.datasets.make_blobs(n_samples=100, n_features=2)
secml_model.fit(X,y)

# Exercise 2

Use the code above as an example to complete the assignment.
* Create a twin-moon sklearn dataset (divided in training and testing)
* Create a SecML wrapper for the newly created classifier
* Fit the classifier using SecML
* Compute the accuracy on the test set

In [ ]:
X, y = sklearn.datasets.make_moons(n_samples=100)
X_t, y_t = sklearn.datasets.make_moons(n_samples=20)

model = SVC()
clf = CClassifierSkLearn(model)
clf.fit(X, y)

y_pred = clf.predict(data_ts.X)

[PyTorch](https://pytorch.org) is a framework for creating deep neural networks, and it is implemented to handle the back-propagation as smooth as possible, by already providing implementations of the most used layers (convolutions, dense, etc.)

A PyTorch neural network is defined as a class that defines its architecture and how if performs the forward pass. This example can be found in the [PyTorch documentation](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

The `Net` class extends the `Module` PyTorch class, and the constructor `__init__` function specifies the architecture. Then, the `forward` function describes how to compute the *logits*, the scores of the classifier.

Now we download the [CIFAR10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) to train the classifier.
To create data that can be consumed by PyTorch, we first need to create a Data Loader that preprocess the input before the forward pass.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

With data and the architecture, we can start the training phase.
We will use [Stochastic Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) to fine-tune the weights of all the layers of the neural network. Lastly, we need a function that quantifies how much the network is performing well while training, that will be used by the optimizer as a guide.
We define a *loss function*, in this case the [Cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy) that quantify the error committed by the neural network.
The larger the loss, the worse the network is behaving. The SGD optimizer will update the model weights to reduce the value of this loss function, and hence creating a network that has a high classification performance.

In [ ]:
import torch.optim as optim

net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
loss_function = nn.CrossEntropyLoss()

We can finally write the PyTorch training loop. Notice that, contrary to scikit-learn, the developer must take into account many technical details before completing the training of a model. On the other hand, PyTorch allows developer to have more control over the shape of the architecture and the training loop.

In [ ]:
for epoch in range(2):  # The network will be more precise if it passes over the dataset more times (beware the overfitting!)

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

In [ ]:
from matplotlib.pyplot import imshow

dataiter = iter(testloader)
images, labels = dataiter.next()

print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))


Now we can wrap the created model inside SecML, through the intended wrapper.

In [ ]:
from secml.ml.classifiers import CClassifierPyTorch
from secml.array import CArray
clf = CClassifierPyTorch(model=net,
                         loss=loss_function,
                         optimizer=optimizer,
                         input_shape=(3, 32, 32),
                         random_state=0,
                         pretrained_classes=CArray(range(10)),
                         pretrained=True)
dataiter = iter(testloader)
images, labels = dataiter.next()
x = CArray(images)
y = clf.predict(x)
print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))